In [1]:
import json
import string
import re
# !pip install pymorphy2
import numpy
import pandas as pd
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Elya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Elya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
#!pip install natasha
from natasha import Doc, Segmenter, NewsNERTagger
from natasha import MorphVocab, NewsEmbedding, NewsMorphTagger
import pymorphy2

  Stored in directory: C:\Users\Elya\AppData\Local\pip\Cache\wheels\f3\f2\66\e9c30d3e9499e65ea2fa0d07c002e64de63bd0adaa49c445bf
Successfully built intervaltree


You are using pip version 19.0.3, however version 21.1.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
file = open("reviews1.json", mode='r', encoding='utf-8') 
read_file = file.read()
# print(read_file)

## Предобработка текста

In [4]:
read_file = read_file.replace('"',"'")
read_file = read_file.replace('«'," ")
read_file = read_file.replace('»'," ")
read_file = read_file.replace("'title': '",'"title": "')
read_file = read_file.replace("'description': '",'"description": "')
read_file = read_file.replace("'\n",'"\n')
read_file = read_file.replace("',\n",'",\n')
read_file = read_file.replace(".",'. ')
read_file = read_file.replace("ё",'е')
read_file = read_file.replace('Ё','Е')
read_file = read_file.replace("!",' ')
read_file = read_file.replace("…",' ')
read_file = read_file.replace("©",' ')
read_file = read_file.replace("—",' ')
read_file = read_file.replace("“",' ')
read_file = read_file.replace("”",' ')

In [5]:
corpus = json.loads(read_file)

In [6]:
docs = []
for i in corpus:
#     print("Запускаем предоработку...")
#     doc = preprocess(i['description'])
#     print("Фильтруем существительные...")
#     doc = filter_noun(doc)
#     print("Удаляем стоп-слова...")
#     doc = remove_stops(doc)
#     print("Лемматизируем текст...")
#     doc = [lemmatize(w) for w in doc]
    doc = i["description"]
    docs.append(doc)
print(docs)

["Главный мультфильм всех времен и народов (но лично для меня - только после  Аладдина ), производящий сильнейшее впечатление. От открывающей сцены, восхода солнца под главную песню Элтона Джона  Circle of life и появления Симбы вплоть до самого финала - рождения нового львенка. О лучшем сложно писать.  Как ни крути, все сводится к восторгам. В мультфильме и правда идеально все:1.  История, вольный ремейк классического  Гамлета , интригует и заставляет переживать до последнего. 2.  Герои, у каждого свой характер, каждый интересен.  Мне в детстве больше всех нравился почему-то Рафики, на своей волне и всегда знающий правильный ответ.  Симба - который перестает бежать от проблем, понимает, что прошлое, хоть и причиняет боль, но важно перестать прятаться и извлечь из него урок, обрести себя и двигаться дальше.  Муфаса - строгий, справедливый и могущественный.  Шрам, который наряду с Джафаром и Урсулой из  Русалочки  - главное злодейское трио, которое никому не превзойти.  Тимон и Пумба и 

In [7]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
embedding = NewsEmbedding()
morph_tagger = NewsMorphTagger(embedding)
ner_tagger = NewsNERTagger(embedding)

analyzer = pymorphy2.MorphAnalyzer()
# doc = Doc(text)

In [8]:
ted_tran = Doc(docs[0])
ted_tran.segment(segmenter)
ted_tran.tag_ner(ner_tagger)
ted_tran.ner.print()

Главный мультфильм всех времен и народов (но лично для меня - только 
после  Аладдина ), производящий сильнейшее впечатление. От открывающей
 сцены, восхода солнца под главную песню Элтона Джона  Circle of life 
                                         PER─────────                 
и появления Симбы вплоть до самого финала - рождения нового львенка. О
            PER──                                                     
 лучшем сложно писать.  Как ни крути, все сводится к восторгам. В 
мультфильме и правда идеально все:1.  История, вольный ремейк 
классического  Гамлета , интригует и заставляет переживать до 
               PER────                                        
последнего. 2.  Герои, у каждого свой характер, каждый интересен.  Мне
 в детстве больше всех нравился почему-то Рафики, на своей волне и 
                                          PER───                   
всегда знающий правильный ответ.  Симба - который перестает бежать от 
проблем, понимает, что прошлое, хоть и пр

Следующая функция работает таким образом:
1. текст сегментируется на токены   
2. на токены накладывается частеречная разметка
3. токены лемматизируются
4. выделяются именованные сущности
5. токены нормализуются
6. обрабатывается нормализация географических объектов

In [9]:
def get_ner(transcript):
    script = Doc(re.sub(r'\((.*?)\)', "", transcript))
    script.segment(segmenter)
    script.tag_morph(morph_tagger)
    for token in script.tokens:
        token.lemmatize(morph_vocab)
    script.tag_ner(ner_tagger)
    for span in script.spans:
        span.normalize(morph_vocab)
    named_ents = [(i.text, i.type, i.normal) for i in script.spans]
    normed_ents = []
    for word, tag, norm in named_ents:
        if len(word.split()) == 1 and tag == "LOC":
            for gram in range(len(analyzer.parse(word))):
                if "Geox" in analyzer.parse(word)[gram].tag:
                    normed_ents.append((analyzer.parse(word)[gram].normal_form))
                    break
                elif gram == len(analyzer.parse(word)) - 1:
                    normed_ents.append((norm.lower().strip(".,!?;-")))
        else:
            normed_ents.append((norm.lower().strip(".,!?;-")))
    return sorted(normed_ents)

In [10]:
named_entities = get_ner(docs[0])
named_entities

['be prepared',
 'disney',
 'disney',
 'hakuna matata',
 'акуна матата',
 'гамлет',
 'джафар',
 'джеймс эрл джонс',
 'джереми айронс',
 'муфасы',
 'мэттью бродерик',
 'пиратов карибское море',
 'пумба',
 'рафики',
 'русалочка',
 'симбу',
 'симбы',
 'тим райс',
 'тимон',
 'урсула',
 'ханс циммер',
 'шрам',
 'шрам',
 'элтон джон',
 'элтон джон']

In [11]:
corpus = []
vocabulary = set()
i = 0

for doc in docs:
    if i % 10 == 0:
        print("Обработано", i, "из 200")
    named_entities = get_ner(doc)
    if named_entities != []:
        corpus.append(named_entities)
        vocabulary.update(named_entities) 
    i += 1

Обработано 0 из 200
Обработано 10 из 200
Обработано 20 из 200
Обработано 30 из 200
Обработано 40 из 200
Обработано 50 из 200
Обработано 60 из 200
Обработано 70 из 200
Обработано 80 из 200
Обработано 90 из 200
Обработано 100 из 200
Обработано 110 из 200
Обработано 120 из 200
Обработано 130 из 200
Обработано 140 из 200
Обработано 150 из 200
Обработано 160 из 200
Обработано 170 из 200
Обработано 180 из 200
Обработано 190 из 200


In [12]:
vocabulary = sorted(set(vocabulary)) 

In [13]:
len(corpus) # изначально было 200 документов

190

In [14]:
len(vocabulary)

429

In [15]:
corpus = [str(x) for x in corpus]
corpus

["['be prepared', 'disney', 'disney', 'hakuna matata', 'акуна матата', 'гамлет', 'джафар', 'джеймс эрл джонс', 'джереми айронс', 'муфасы', 'мэттью бродерик', 'пиратов карибское море', 'пумба', 'рафики', 'русалочка', 'симбу', 'симбы', 'тим райс', 'тимон', 'урсула', 'ханс циммер', 'шрам', 'шрам', 'элтон джон', 'элтон джон']",
 "['дальше симба', 'дисней', 'земля', 'король', 'король', 'пап', 'прайд львамуфасыродился', 'пумбы', 'симба', 'симбы', 'симбы', 'тимон', 'хакуна матата', 'шрам']",
 "['animal planet', 'африка', 'джеймс эрл джонс', 'джон фавро', 'джон фавро', 'джони', 'дисней', 'начитавшись', 'ханс цимммер']",
 "['дисней', 'дисней', 'дисней', 'дисней', 'кения', 'кения', 'король', 'москва', 'нала', 'налы', 'покахонтас', 'ээрон', 'ээрон блейз', 'ээрон блейз']",
 "['африка', 'африка', 'величественных львы', 'короли', 'король', 'король', 'король-лев муфасы', 'симба', 'симба', 'симбе', 'симбой', 'симбы', 'симбы', 'шрам']",
 "['андрей гаврилов', 'андрей гаврилов', 'андрей гаврилов', 'вупи 

## Кластеризация 

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.cluster import KMeans

### CountVectorizer - мешок слов

Смысл мешка слов в том, чтобы посчитать, сколько раз каждое слово встретилось в документе


### TfidfTransformer

Смысл метода tf-idf в том, чтобы присвоить наибольшие коэффициенты наиболее "значимым словам", или словам, характерным для кнкретного документа


In [17]:
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
                 ('tfid', TfidfTransformer())]).fit(corpus)

X = pipe.fit_transform(corpus)
km = KMeans(n_clusters=15, init='k-means++', max_iter=600, 
            algorithm="full", precompute_distances=True)

In [18]:
km.fit(X)

c:\users\elya\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\cluster\_kmeans.py:970: FutureWarning: 'precompute_distances' was deprecated in version 0.23 and will be removed in 0.25. It has no effect
  "effect", FutureWarning)


KMeans(algorithm='full', max_iter=600, n_clusters=15, precompute_distances=True)

Силуэт: показывает, насколько объект похож на свой кластер относительно других кластеров; если значение стремится к 1 - хорошее разбиение, если к -1 - плохое, если в районе 0 - кластеры пересекаются

Индекс Дэвиcа-Болдуина: оценивает расстояние от объекта кластера до центроида и расстояние между центроидами; чем ниже, тем лучше разбиение

In [19]:
print(metrics.silhouette_score(X, km.labels_, sample_size=1000))
print(metrics.davies_bouldin_score(X.toarray(), km.labels_))

0.06262982505406169
2.736034086613578


Была проведите кластеризация документов корпуса по именованным сущностям.
Для этого отобраны только те документы, в которых именованные сущности есть (190 из 200). По оценкам силуэта можно сказать, что кластеры пересекаются, индекс Дэвиcа-Болдуина равен 2, это довольно небольшое значение, а, значит, разбиение хорошее.